In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

#Jets adding
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [13]:
#import combined data from csv
#data = pd.read_csv("E:\\6242Project\\FeaturesDataset\\philly_property_data.csv")
#data = pd.read_csv("E:\\6242Project\\FeaturesDataset\\washington_property_data.csv")
#data = pd.read_csv("E:\\6242Project\\FeaturesDataset\\philly_enriched.csv")
data = pd.read_csv("C:\\Users\\easto\\OneDrive\\Desktop\\College\\Masters\\Semester 2\\Data and Visual Analytics\\Local Project Data\\philly_final_dataset.csv")



In [ ]:
y = data['sale_price']
data_Features = data.drop(columns=['market_value', 'sale_price'])
#data_hot = data_Features.drop(columns=['sale_date'])
#data_hot = data_Features.drop(columns=['sale_date','view_scale','heater_scale','central_air','garage_spaces','has_central_air', 'has_type_heater'])
data_hot = data_Features
data_hot = pd.get_dummies(data_hot, drop_first=True)


In [15]:
#train test split
#X_train, X_test, y_train, y_test = train_test_split(data10_hot, y, test_size=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(data_hot, y, test_size=0.3, random_state=1)

# Standardize full feature set for Elastic Net variable selection
scaler_full = StandardScaler()
X_train_scaled_full = scaler_full.fit_transform(X_train)
X_test_scaled_full = scaler_full.transform(X_test)

In [16]:
# Variable selection with Elastic Net
elasticNet = ElasticNetCV(l1_ratio=0.5, cv=5)
elasticNet.fit(X_train_scaled_full, y_train)

# Get the selected features
coefficients = elasticNet.coef_
selected_features = data_hot.columns[coefficients != 0]
print("Selected features:")
print(selected_features)

Selected features:
Index(['zip_code', 'view_scale', 'heater_scale', 'exterior_condition',
       'interior_condition', 'central_air', 'basement_scale', 'garage_spaces',
       'number_of_bedrooms', 'number_of_bathrooms', 'number_stories',
       'total_livable_area', 'total_area', 'year_built', 'year_built_estimate',
       'has_basements', 'has_central_air', 'has_type_heater',
       'is_nominal_sale', 'market_value', 'active_inventory', 'new_listings',
       'pending_sales', 'homes_sold', 'sale_to_list_ratio',
       'percent_above_list', 'avg_days_on_market', 'crimes_per_100k',
       'avg_list_price_per_sqft', 'avg_listed_price', 'avg_price_per_sqft',
       'avg_price_sold', 'sale_day', 'sale_month', 'sale_year', 'est_list_day',
       'est_list_month', 'est_list_year', 'Est TOM in Days'],
      dtype='object')


In [17]:
print("--------------------------------------End of Variable Selection--------------------------------------")

--------------------------------------End of Variable Selection--------------------------------------


In [18]:
#Instiantiate model
LMmodel = LinearRegression()

#fit model to training data. Using only selected features
X_train_selected = X_train[selected_features]

LMmodel.fit(X_train_selected, y_train)

#predict on test data
x_test_selected = X_test[selected_features]
y_pred = LMmodel.predict(x_test_selected)

print("Predictions:")
print(y_pred)

#calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred)
print(R2)

#Access coefficients
coefficients = LMmodel.coef_
print("Coefficients:")
print(coefficients)

#Access intercept
intercept = LMmodel.intercept_
print("Intercept:")
print(intercept)

Predictions:
[ -23073.625  255150.125  605125.25  ... 1741806.875  128866.25
  692749.625]
RMSE:
845813.8344271103
R2:
0.4352398999560986
Coefficients:
[-7.32648937e+02 -3.97354940e+03 -4.16405744e+03 -2.87525640e+02
  6.24255807e+03  1.24663945e+05  1.67568692e+04  9.94961220e+04
  3.83111971e+04  6.63530609e+04  3.16460987e+05  6.56967124e+00
  6.77661602e-03 -2.84602114e+02  1.91866641e+04 -3.15411004e+05
 -5.32375422e+03 -1.32327266e+05 -2.98814859e+04  4.26624565e-01
 -2.32907283e+03  3.21438582e+03  9.39739345e+03 -1.10656171e+04
 -3.69432030e+05  1.79934865e+04 -1.10128523e+04 -1.12484842e+11
 -5.23769281e+02  3.19405469e-01  1.17831697e+03 -3.84833731e-01
  1.42888089e+04  1.60775436e+03  1.67309291e+04 -1.05421756e+04
  2.03354624e+03 -1.28212393e+04  3.12975465e+05]
Intercept:
614359763828335.9


In [19]:
#Instantiating Random Forest Model
RFmodel = RandomForestRegressor(n_estimators=50, random_state=1,n_jobs=-1 ,verbose=1)

#RF model fit
RFmodel.fit(X_train, y_train)

#Sanity check
print("Progress...")

#Predicting on test data
y_pred_RF = RFmodel.predict(X_test)
print("Predictions:")
print(y_pred_RF)

# Calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_RF))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred_RF)
print(R2)

# Access feature importances (since Random Forest models provide feature importance)
feature_importances = RFmodel.feature_importances_
print("Feature Importances:")
print(feature_importances)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s


Progress...
Predictions:
[2609695.66  101299.1   355752.02 ... 2299788.12  192269.96  534931.74]
RMSE:
651848.9288046993
R2:
0.6645650198347236
Feature Importances:
[1.23824257e-02 9.12286307e-04 2.26538061e-03 5.78246598e-04
 5.33576108e-04 2.04984943e-04 1.03115028e-03 3.17492949e-03
 2.28225747e-02 1.07923450e-02 1.72321512e-01 1.42507552e-01
 1.34982167e-02 2.64347427e-02 2.12343039e-04 3.18755905e-05
 2.40119466e-04 6.12760944e-04 3.90099477e-03 4.52127389e-01
 6.62610428e-03 6.08785903e-03 1.50710429e-02 1.53812411e-03
 6.18518245e-03 6.63041808e-03 3.03163050e-03 0.00000000e+00
 6.48057333e-03 5.92648625e-03 5.65452567e-03 2.43301318e-03
 2.92338662e-03 2.18813404e-02 2.68427391e-02 1.26971922e-04
 4.23777145e-03 9.10765179e-03 2.04252180e-03 5.87251288e-04]


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.7s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    0.0s finished


In [20]:
# Instantiate Gradient Boosting model

'''
GBmodel = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.15,
    max_depth=4,
    random_state=1,
    verbose=1
)
'''
GBmodel = GradientBoostingRegressor(
    subsample= 1.0, n_estimators= 500,
    min_samples_split= 2, min_samples_leaf= 1,
    max_features= None, max_depth= 4, learning_rate= 0.15
)

# Fit the model on untransformed target
GBmodel.fit(X_train, y_train)

# Predict on test data
y_pred_GB = GBmodel.predict(X_test)

# Evaluate performance
RMSE_GB = np.sqrt(mean_squared_error(y_test, y_pred_GB))
R2_GB = r2_score(y_test, y_pred_GB)

print("Predictions:")
print(y_pred_GB)
print("RMSE (Gradient Boosting):", RMSE_GB)
print("R² (Gradient Boosting):", R2_GB)

# Access feature importances
feature_importances_GB = GBmodel.feature_importances_
print("Feature Importances (Gradient Boosting):")
print(feature_importances_GB)

Predictions:
[1908489.28057221   90702.66985398  391082.66783975 ... 2081460.45358845
  184379.51783614  453965.32259484]
RMSE (Gradient Boosting): 690812.7493570889
R² (Gradient Boosting): 0.6232657316963437
Feature Importances (Gradient Boosting):
[1.84606240e-02 1.70624092e-04 1.38825407e-03 2.78793858e-04
 6.22624546e-04 2.34726385e-03 1.57167486e-04 1.38438353e-03
 6.12629551e-02 1.78957787e-02 1.52909853e-01 7.84399568e-02
 1.45357349e-02 1.11634125e-02 1.17502304e-04 2.61749205e-05
 6.20522124e-04 8.79335800e-04 3.94719311e-03 5.44584571e-01
 1.44482018e-02 3.14957108e-03 3.46868591e-04 5.86768818e-04
 2.02696156e-03 1.07130176e-02 1.42071678e-03 0.00000000e+00
 2.92711088e-03 2.92173908e-03 9.35105594e-04 1.10331293e-03
 1.66706206e-03 1.44941692e-02 2.00757096e-02 8.40941417e-05
 2.78497092e-03 8.19704265e-03 5.06989162e-04 4.17861743e-04]


In [21]:
# Define parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Instantiate base model
gbr = GradientBoostingRegressor(random_state=1)

# Set up randomized search
rand_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=50,               # Number of random combinations to try
    scoring='r2',            # Optimize for R² score
    cv=3,                    # 3-fold cross-validation
    verbose=2,
    n_jobs=-1,               # Use all available cores
    random_state=42
)

# Fit to training data
rand_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters Found:")
print(rand_search.best_params_)
print("Best Cross-Validated R² Score:")
print(rand_search.best_score_)

# Evaluate on test set
best_gbr = rand_search.best_estimator_
y_pred_best = best_gbr.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2_best = r2_score(y_test, y_pred_best)

print("\nTest Set Evaluation")
print("RMSE:", rmse_best)
print("R²:", r2_best)

#with open("modelSale.pkl", "wb") as f:
    #import pickle
joblib.dump(best_gbr, 'modelSale.pkl')


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters Found:
{'subsample': 1.0, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 5, 'learning_rate': 0.05}
Best Cross-Validated R² Score:
0.6027468648304829

Test Set Evaluation
RMSE: 669651.287924494
R²: 0.6459929959238864


['modelSale.pkl']